# Gurobi CSP Solver

Ce notebook utilise Gurobi pour résoudre les problèmes de satisfaction de contraintes (CSP) et trouver toutes les solutions possibles afin de vérifier les solutions obtenues par CPSolver.

## Installation et utilisation

Pour installer les packages Julia nécessaires :
```julia
import Pkg; Pkg.add(["JuMP", "Gurobi", "MathOptInterface"])
```

Pour lancer le serveur Jupyter :
```bash
cd /home/madoff/CPSolver && /home/madoff/venv/bin/jupyter notebook --no-browser --port=8888
```

In [1]:
# import Pkg; Pkg.add(["JuMP", "Gurobi", "MathOptInterface"])

In [ ]:
# Configuration de l'environnement Gurobi AVANT l'import des packages
# Ces variables doivent être définies avant que Julia charge Gurobi

# Configuration de l'environnement Gurobi
# ENV["GUROBI_HOME"] = "/home/madoff/.julia/artifacts/11980007f4c80fcdcbdd8e7b8dafaa8657dcead9"
# ENV["GRB_LICENSE_FILE"] = "/home/madoff/.gurobi/gurobi.lic"

# Configuration pour éviter les problèmes de connectivité
# ENV["GRB_WLSACCESSID"] = "891e8c63-a77b-40dc-ba42-f8b6d8314067"
# ENV["GRB_WLSSECRET"] = "493b4494-24c0-407f-a7c2-75550d53c44d"
# ENV["GRB_LICENSEID"] = "2710287"

# println("GUROBI_HOME: ", ENV["GUROBI_HOME"])
# println("GRB_LICENSE_FILE: ", ENV["GRB_LICENSE_FILE"])
# println("GRB_WLSACCESSID: ", ENV["GRB_WLSACCESSID"])
# println("GRB_LICENSEID: ", ENV["GRB_LICENSEID"])

In [3]:
# Import des packages nécessaires
using JuMP
using Gurobi
using MathOptInterface
using Printf
using Graphs
using GraphPlot
using Plots
using DelimitedFiles

## Structures de données

In [4]:
# Structures de données pour les instances CSP
struct CSPInstance
    name::String
    num_variables::Int
    domain_size::Int
    num_constraints::Int
    domains::Vector{Tuple{Int, Int, Int}}  # (var_id, min_val, max_val)
    constraints::Vector{Tuple{Int, Int, Vector{Tuple{Int, Int}}}}  # (var1, var2, allowed_pairs)
    
    function CSPInstance(name::String, num_variables::Int, domain_size::Int, num_constraints::Int,
                        domains::Vector{Tuple{Int, Int, Int}}, 
                        constraints::Vector{Tuple{Int, Int, Vector{Tuple{Int, Int}}}})
        new(name, num_variables, domain_size, num_constraints, domains, constraints)
    end
end

struct CSPSolution
    instance_name::String
    num_variables::Int
    domain_size::Int
    solutions::Vector{Dict{Int, Int}}  # Dict{var_id => value}
    solve_time::Float64
    num_nodes::Int
    gap::Float64
    
    function CSPSolution(instance_name::String, num_variables::Int, domain_size::Int,
                        solutions::Vector{Dict{Int, Int}}, solve_time::Float64, 
                        num_nodes::Int, gap::Float64)
        new(instance_name, num_variables, domain_size, solutions, solve_time, num_nodes, gap)
    end
end

## Lecteur d'instances CSP

In [5]:
# Fonction de lecture d'instances CSP
function parse_csp_instance(filename::String)::CSPInstance
    """
    Parse une instance CSP au format standard.
    """
    name = basename(filename)
    
    open(filename, "r") do file
        lines = readlines(file)
        
        num_variables = 0
        domain_size = 0
        num_constraints = 0
        domains = Tuple{Int, Int, Int}[]
        constraints = Tuple{Int, Int, Vector{Tuple{Int, Int}}}[]
        
        line_idx = 1
        
        # Ignorer les commentaires au début
        while line_idx <= length(lines) && (isempty(strip(lines[line_idx])) || startswith(strip(lines[line_idx]), "#"))
            line_idx += 1
        end
        
        # Lire le nombre de variables
        if line_idx <= length(lines)
            num_variables = parse(Int, strip(lines[line_idx]))
            line_idx += 1
        end
        
        # Ignorer les lignes vides et commentaires
        while line_idx <= length(lines) && (isempty(strip(lines[line_idx])) || startswith(strip(lines[line_idx]), "#"))
            line_idx += 1
        end
        
        # Lire les domaines des variables
        while line_idx <= length(lines) && !isempty(strip(lines[line_idx])) && !startswith(strip(lines[line_idx]), "#")
            parts = split(strip(lines[line_idx]))
            if length(parts) >= 3
                var_id = parse(Int, parts[1])
                min_val = parse(Int, parts[2])
                max_val = parse(Int, parts[3])
                push!(domains, (var_id, min_val, max_val))
                domain_size = max(domain_size, max_val)
            end
            line_idx += 1
        end
        
        # Ignorer les lignes vides et commentaires
        while line_idx <= length(lines) && (isempty(strip(lines[line_idx])) || startswith(strip(lines[line_idx]), "#"))
            line_idx += 1
        end
        
        # Lire le nombre de contraintes
        if line_idx <= length(lines)
            num_constraints = parse(Int, strip(lines[line_idx]))
            line_idx += 1
        end
        
        # Ignorer les lignes vides et commentaires
        while line_idx <= length(lines) && (isempty(strip(lines[line_idx])) || startswith(strip(lines[line_idx]), "#"))
            line_idx += 1
        end
        
        # Lire les contraintes
        while line_idx <= length(lines) && !isempty(strip(lines[line_idx]))
            line = strip(lines[line_idx])
            if !startswith(line, "#")
                parts = split(line)
                if length(parts) >= 2
                    var1 = parse(Int, parts[1])
                    var2 = parse(Int, parts[2])
                    
                    # Parser les paires de valeurs autorisées
                    allowed_pairs = Tuple{Int, Int}[]
                    for i in 3:length(parts)
                        pair_str = parts[i]
                        if startswith(pair_str, "(") && endswith(pair_str, ")")
                            pair_str = pair_str[2:end-1]  # Enlever les parenthèses
                            pair_parts = split(pair_str, ",")
                            if length(pair_parts) == 2
                                val1 = parse(Int, pair_parts[1])
                                val2 = parse(Int, pair_parts[2])
                                push!(allowed_pairs, (val1, val2))
                            end
                        end
                    end
                    
                    if !isempty(allowed_pairs)
                        push!(constraints, (var1, var2, allowed_pairs))
                    end
                end
            end
            line_idx += 1
        end
        
        return CSPInstance(name, num_variables, domain_size, num_constraints, domains, constraints)
    end
end

parse_csp_instance (generic function with 1 method)

## Résolution CSP avec Gurobi et JuMP

In [6]:
function verify_csp_solution(instance::CSPInstance, solution::Dict{Int, Int})::Bool
    """
    Vérifie qu'une solution satisfait toutes les contraintes CSP.
    """
    for (var1, var2, allowed_pairs) in instance.constraints
        val1 = solution[var1]
        val2 = solution[var2]
        if !((val1, val2) in allowed_pairs)
            return false
        end
    end
    return true
end

verify_csp_solution (generic function with 1 method)

In [7]:
# Solveur CSP avec Gurobi - Trouve toutes les solutions
function solve_csp_instance(instance::CSPInstance; max_solutions::Int=2000)::CSPSolution
    """
    Résout une instance CSP en utilisant Gurobi et JuMP pour trouver TOUTES les solutions.
    Version optimisée utilisant une approche hybride efficace.
    """
    println("=== Résolution de l'instance: ", instance.name)
    
    all_solutions = Dict{Int, Int}[]
    total_solve_time = 0.0
    total_nodes = 0
    max_gap = 0.0
    
    # Créer le modèle JuMP avec Gurobi (UNE SEULE FOIS)
    model = Model(Gurobi.Optimizer)
    
    # Variables binaires pour chaque assignation possible
    # x[var, val] = 1 si la variable var prend la valeur val
    @variable(model, x[0:instance.num_variables-1, 1:instance.domain_size], Bin)
    
    # Contraintes : chaque variable doit prendre exactement une valeur
    for var in 0:instance.num_variables-1
        @constraint(model, sum(x[var, val] for val in 1:instance.domain_size) == 1)
    end
    
    # Contraintes : respecter les contraintes CSP
    for (var1, var2, allowed_pairs) in instance.constraints
        # Créer une contrainte pour chaque paire non autorisée
        forbidden_pairs = []
        for val1 in 1:instance.domain_size
            for val2 in 1:instance.domain_size
                if !((val1, val2) in allowed_pairs)
                    push!(forbidden_pairs, (val1, val2))
                end
            end
        end
        
        # Interdire les paires non autorisées
        for (val1, val2) in forbidden_pairs
            @constraint(model, x[var1, val1] + x[var2, val2] <= 1)
        end
    end
    
    # Objectif : maximiser le nombre de variables assignées (toujours égal au nombre de variables)
    @objective(model, Max, sum(x[var, val] for var in 0:instance.num_variables-1, val in 1:instance.domain_size))
    
    # Configuration Gurobi optimisée
    set_optimizer_attribute(model, "OutputFlag", 1)  # Afficher les informations de progression
    set_optimizer_attribute(model, "TimeLimit", 300)  # 5 minutes max
    set_optimizer_attribute(model, "MIPGap", 0.0)     # Solution exacte
    
    # Configuration pour trouver plusieurs solutions
    set_optimizer_attribute(model, "PoolSearchMode", 1)  # Mode 1: trouve plusieurs solutions optimales
    set_optimizer_attribute(model, "PoolSolutions", max_solutions)  # Nombre maximum de solutions à stocker
    
    println("Configuration optimisée:")
    println("  - Mode de recherche: Multiple solutions optimales")
    println("  - Nombre maximum de solutions: ", max_solutions)
    println("  - Temps limite: 300 secondes")
    
    # Résoudre (UNE SEULE FOIS)
    println("\\nDébut de la résolution...")
    optimize!(model)
    
    # Vérifier le statut
    status = termination_status(model)
    println("Statut de résolution: ", status)
    
    if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED || status == MOI.TIME_LIMIT
        # Extraire la solution optimale trouvée
        println("Extraction de la solution optimale...")
        
        try
            solution = Dict{Int, Int}()
            for var in 0:instance.num_variables-1
                for val in 1:instance.domain_size
                    if value(x[var, val]) > 0.5
                        solution[var] = val
                        break
                    end
                end
            end
            
            if !isempty(solution)
                push!(all_solutions, solution)
                println("Solution optimale extraite: ", solution)
                
                # Vérifier la solution
                if verify_csp_solution(instance, solution)
                    println("✓ Solution optimale vérifiée!")
                else
                    println("✗ Erreur dans la solution optimale!")
                end
            end
        catch e
            println("Erreur lors de l'extraction de la solution optimale: ", e)
        end
        
        # Pour trouver toutes les solutions, utiliser une approche différente
        # Créer un nouveau modèle avec contraintes d'exclusion
        println("\\nRecherche des solutions supplémentaires...")
        
        iteration = 0
        max_iterations = min(max_solutions * 2, 2000)
        
        while length(all_solutions) < max_solutions && iteration < max_iterations
            iteration += 1
            
            # Créer un nouveau modèle pour cette itération
            model_iter = Model(Gurobi.Optimizer)
            
            # Variables binaires
            @variable(model_iter, x_iter[0:instance.num_variables-1, 1:instance.domain_size], Bin)
            
            # Contraintes : chaque variable doit prendre exactement une valeur
            for var in 0:instance.num_variables-1
                @constraint(model_iter, sum(x_iter[var, val] for val in 1:instance.domain_size) == 1)
            end
            
            # Contraintes : respecter les contraintes CSP
            for (var1, var2, allowed_pairs) in instance.constraints
                forbidden_pairs = []
                for val1 in 1:instance.domain_size
                    for val2 in 1:instance.domain_size
                        if !((val1, val2) in allowed_pairs)
                            push!(forbidden_pairs, (val1, val2))
                        end
                    end
                end
                
                for (val1, val2) in forbidden_pairs
                    @constraint(model_iter, x_iter[var1, val1] + x_iter[var2, val2] <= 1)
                end
            end
            
            # Contraintes d'exclusion : interdire les solutions déjà trouvées
            for existing_solution in all_solutions
                exclusion_terms = []
                for var in 0:instance.num_variables-1
                    assigned_value = existing_solution[var]
                    push!(exclusion_terms, x_iter[var, assigned_value])
                end
                @constraint(model_iter, sum(exclusion_terms) <= instance.num_variables - 1)
            end
            
            # Objectif
            @objective(model_iter, Max, sum(x_iter[var, val] for var in 0:instance.num_variables-1, val in 1:instance.domain_size))
            
            # Configuration rapide
            set_optimizer_attribute(model_iter, "OutputFlag", 0)
            set_optimizer_attribute(model_iter, "TimeLimit", 10)
            set_optimizer_attribute(model_iter, "MIPGap", 0.0)
            
            # Résoudre
            optimize!(model_iter)
            
            status_iter = termination_status(model_iter)
            
            if status_iter == MOI.OPTIMAL || status_iter == MOI.LOCALLY_SOLVED
                # Extraire la nouvelle solution
                new_solution = Dict{Int, Int}()
                for var in 0:instance.num_variables-1
                    for val in 1:instance.domain_size
                        if value(x_iter[var, val]) > 0.5
                            new_solution[var] = val
                            break
                        end
                    end
                end
                
                # Vérifier que c'est une nouvelle solution
                is_new = true
                for existing_solution in all_solutions
                    if new_solution == existing_solution
                        is_new = false
                        break
                    end
                end
                
                if is_new
                    push!(all_solutions, new_solution)
                    println("Solution ", length(all_solutions), " trouvée: ", new_solution)
                    
                    # Vérifier la solution
                    if verify_csp_solution(instance, new_solution)
                        println("✓ Solution ", length(all_solutions), " vérifiée!")
                    else
                        println("✗ Erreur dans la solution ", length(all_solutions), "!")
                    end
                else
                    println("Solution dupliquée trouvée, arrêt de la recherche")
                    break
                end
            else
                println("Aucune nouvelle solution trouvée (status: ", status_iter, ")")
                break
            end
        end
        
        println("Recherche terminée après ", iteration, " itérations")
    else
        println("Aucune solution trouvée (status: ", status, ")")
    end
    
    # Récupérer les paramètres de résolution
    total_solve_time = try
        solve_time(model)
    catch
        0.0
    end
    
    total_nodes = try
        node_count(model)
    catch
        0
    end
    
    max_gap = try
        relative_gap(model)
    catch
        0.0
    end
    
    println("\\n=== Résumé de la recherche de toutes les solutions (OPTIMISÉE CORRIGÉE) ===")
    println("Solutions trouvées: ", length(all_solutions))
    println("Temps total de résolution: ", round(total_solve_time, digits=4), " secondes")
    println("Nombre total de nœuds explorés: ", total_nodes)
    println("Gap relatif maximum: ", round(max_gap, digits=6))
    
    return CSPSolution(instance.name, instance.num_variables, instance.domain_size, all_solutions, total_solve_time, total_nodes, max_gap)
end

solve_csp_instance (generic function with 1 method)

In [8]:
# Fonction d'écriture des solutions au format .sol
function write_csp_solution(solution::CSPSolution, filename::String)
    """
    Écrit la solution CSP dans un fichier au format .sol avec les paramètres de résolution.
    """
    open(filename, "w") do file
        println(file, "# Solution for ", solution.instance_name)
        println(file, "# Variables: ", solution.num_variables, ", Domain size: ", solution.domain_size)
        println(file, "# Number of solutions found: ", length(solution.solutions))
        println(file, "")
        println(file, "# Resolution parameters:")
        println(file, "# Gurobi time: ", round(solution.solve_time, digits=4), " seconds")
        println(file, "# Number of nodes: ", solution.num_nodes)
        println(file, "# Relative gap: ", round(solution.gap, digits=6))
        println(file, "")
        
        for (i, sol) in enumerate(solution.solutions)
            println(file, "# Solution ", i)
            var_assignments = []
            for var in 0:solution.num_variables-1
                value = sol[var]
                push!(var_assignments, "$(var)=$(value)")
            end
            println(file, join(var_assignments, " "))
            println(file, "")
        end
    end
    println("Solution sauvegardée dans: ", filename)
end

write_csp_solution (generic function with 1 method)

In [9]:
# Fonction principale de résolution CSP
function solve_csp_file(instance_name::String; 
                        instances_dir::String="../instances/instances", 
                        solutions_dir::String="../solutions/solutions")
    """
    Méthode principale pour résoudre une instance CSP.
    Charge l'instance, résout avec Gurobi et sauvegarde la solution.
    """
    println("="^60)
    println("RÉSOLUTION D'INSTANCE CSP AVEC GUROBI")
    println("="^60)
    
    # Chemin vers l'instance
    instance_path = joinpath(instances_dir, instance_name)
    if !isfile(instance_path)
        println("Erreur: Fichier d'instance non trouvé: ", instance_path)
        return nothing
    end
    
    # Charger l'instance
    println("\\n1. Chargement de l'instance...")
    instance = parse_csp_instance(instance_path)
    
    # Afficher les détails de l'instance
    println("\\n2. Analyse de l'instance...")
    print_csp_instance_details(instance)
    
    # Résoudre l'instance
    println("\\n3. Résolution...")
    solution = solve_csp_instance(instance)
    
    # Sauvegarder la solution
    println("\\n4. Sauvegarde de la solution...")
    solution_filename = replace(instance_name, ".csp" => "_gb.sol")
    solution_path = joinpath(solutions_dir, solution_filename)
    
    # Créer le répertoire s'il n'existe pas
    mkpath(dirname(solution_path))
    
    write_csp_solution(solution, solution_path)
    
    # Résumé final
    println("\\n" * "="^60)
    println("RÉSUMÉ DE LA RÉSOLUTION")
    println("="^60)
    println("Instance: ", instance.name)
    println("Variables: ", instance.num_variables)
    println("Domaine: 1 à ", instance.domain_size)
    println("Contraintes: ", instance.num_constraints)
    println("Solutions trouvées: ", length(solution.solutions))
    println("Temps de résolution: ", round(solution.solve_time, digits=4), " secondes")
    println("Nœuds explorés: ", solution.num_nodes)
    println("Gap relatif: ", round(solution.gap, digits=6))
    println("Fichier de solution: ", solution_path)
    
    return solution
end

function print_csp_instance_details(instance::CSPInstance)
    """
    Affiche les détails d'une instance CSP.
    """
    println("=== Détails de l'instance CSP ===")
    println("Nom: ", instance.name)
    println("Nombre de variables: ", instance.num_variables)
    println("Taille du domaine: ", instance.domain_size)
    println("Nombre de contraintes: ", instance.num_constraints)
    
    # Calculer les statistiques
    total_allowed_pairs = sum(length(pairs) for (_, _, pairs) in instance.constraints)
    avg_allowed_pairs = total_allowed_pairs / length(instance.constraints)
    
    println("Nombre total de paires autorisées: ", total_allowed_pairs)
    println("Nombre moyen de paires par contrainte: ", round(avg_allowed_pairs, digits=2))
    
    # Afficher quelques contraintes
    println("\\nPremières contraintes:")
    for (i, (var1, var2, pairs)) in enumerate(instance.constraints[1:min(3, length(instance.constraints))])
        println("  Contrainte ", i, ": var", var1, " et var", var2, " (", length(pairs), " paires autorisées)")
    end
    
    if length(instance.constraints) > 3
        println("  ... et ", length(instance.constraints) - 3, " autres contraintes")
    end
end

print_csp_instance_details (generic function with 1 method)

## Résolution des instances de intances/instances

In [10]:
# Charger l'instance equality_example.csp
instance_path = "../instances/instances/equality_example.csp"
solution_path = "../solutions/solutions/equality_example_gb.sol"
instance = parse_csp_instance(instance_path)

# Résoudre avec la fonction corrigée
solution = solve_csp_instance(instance)

write_csp_solution(solution, solution_path)

println("Nombre de solutions trouvées: ", length(solution.solutions))
for (i, sol) in enumerate(solution.solutions)
    println("Solution ", i, ": ", sol)
end

=== Résolution de l'instance: equality_example.csp
Set parameter Username
Set parameter LicenseID to value 2709359
Academic license - for non-commercial use only - expires 2026-09-16
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0
Set parameter PoolSearchMode to value 1
Set parameter PoolSolutions to value 2000
Configuration optimisée:
  - Mode de recherche: Multiple solutions optimales
  - Nombre maximum de solutions: 2000
  - Temps limite: 300 secondes
\nDébut de la résolution...
Set parameter MIPGap to value 0
Set parameter PoolSearchMode to value 1
Set parameter PoolSolutions to value 2000
Set parameter TimeLimit to value 300
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default p

In [11]:
# boucle de résolution sur toutes les instances du dossier instances/instances
for instance_name in readdir("../instances/instances")
    println("Résolution de l'instance: ", instance_name)
    solve_csp_file(instance_name)
end

Résolution de l'instance: equality_example.csp
RÉSOLUTION D'INSTANCE CSP AVEC GUROBI
\n1. Chargement de l'instance...
\n2. Analyse de l'instance...
=== Détails de l'instance CSP ===
Nom: equality_example.csp
Nombre de variables: 4
Taille du domaine: 3
Nombre de contraintes: 4
Nombre total de paires autorisées: 12
Nombre moyen de paires par contrainte: 3.0
\nPremières contraintes:
  Contrainte 1: var2 et var3 (3 paires autorisées)
  Contrainte 2: var0 et var1 (3 paires autorisées)
  Contrainte 3: var1 et var3 (3 paires autorisées)
  ... et 1 autres contraintes
\n3. Résolution...
=== Résolution de l'instance: equality_example.csp
Set parameter Username
Set parameter LicenseID to value 2709359
Academic license - for non-commercial use only - expires 2026-09-16
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0
Set parameter PoolSearchMode to value 1
Set parameter PoolSolutions to value 2000
Configuration optimisée:
  - Mode de recherch

Excessive output truncated after 524288 bytes.

606 trouvée: Dict(5 => 9, 8 => 2, 1 => 3, 0 => 8, 6 => 11, 9 => 6, 3 => 4, 7 => 5, 4 => 7, 2 => 1, 10 => 10)
✓ Solution 606 vérifiée!
Set parameter Username
Set parameter LicenseID to value 2709359
Academic license - for non-commercial use only - expires 2026-09-16
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 10
Solution 607 trouvée: Dict(5 => 5, 8 => 9, 1 => 4, 0 => 11, 6 => 8, 9 => 6, 3 => 10, 7 => 1, 4 => 3, 2 => 7, 10 => 2)
✓ Solution 607 vérifiée!
Set parameter Username
Set parameter LicenseID to value 2709359
Academic license - for non-commercial use only - expires 2026-09-16
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 10
Solution 608 trouvée: Dict(5 => 